# Homework 2: SQL in Python (Data and Databases 2021)

The next four homework assignments will be in this iPython notebook format. Read the instructions as you go, and type in the necessary code--then run the cells to see if you got it right. Don't worry if you are new to python for this assignment--the only "code" you will be writing are SQL statements that you will paste into each myquery variable.


## The Challenges

For this homework we will be using the tables we imported from the [UN data site](http://data.un.org/Explorer.aspx). You should already have `wind` and `solar` installed. As you will see, this homework asks that you find one more table of your choice from the UN's Energy statistics an import it into the the same database. [I have posted the instructions here](http://floatingmedia.com/columbia/csv_to_sql.html)

These are relatively simple CSV files--the only really important columns are `country`, `year` and `usage`. But the tricky part is that most countries have multiple entries: one for every year the UN has been gathering data on that country. So most of these queries are going to ask you to get electrical usage results based on country and year.

In the questions below, we focus on exploratory queries--with an emphasis on aggregates and joins. Read the questions carefully, and then write SQL statements that will get you the correct results. (You will want to explore tables and test out queries directly in the shell before you paste them into Python--the error handling for SQL in Python can be confusing, and there should be no errors in the prewritten Python parts of this document. Also, sometimes an error in Python will break your connection with the database--see conn.rollback() below for that solution.


First things first: Connect to database! 
(Make sure the cell is selected, then go to the cell menubar and choose "Run Cells")

In [3]:
import pg8000.dbapi
# use whatever username and other information you used in class to connect
conn = pg8000.dbapi.connect(database="undata",user=" ")
print(type(conn))

<class 'pg8000.dbapi.Connection'>


If the connection isn't working make sure that you have typed in the name of the database correctly, and your username as well. 

If you get an error when running one of the SQL cells below, run this conn.rollback() cell to restart your connection.

In [4]:
conn.rollback()

## Question 1: Simple(ish) Query 

Create a statement that searches for the top 15 countries for solar energy usage in 2014. So figure out first what table you are searching and what columns you need (you can figure out the columns by looking at the sample results below). Then you need to test by year so you only get results from 2014, and order it correctly.

```
Germany 36056.0 mil
United States 25764.0 mil
Japan 22952.0 mil
Italy 22306.0 mil
China 15189.0 mil
Spain 13673.0 mil
France 6392.0 mil
India 5020.0 mil
United Kingdom 4054.0 mil
Australia 4010.0 mil
Greece 3792.0 mil
Belgium 2886.0 mil
Korea, Republic of 2557.0 mil
Czechia 2123.0 mil
Canada 2120.0 mil
```

In [15]:
cursor = conn.cursor()
myquery = "SELECT country, usage FROM solar WHERE year = '2014' ORDER BY usage DESC LIMIT 15;"
cursor.execute(myquery)
for row in cursor:
    print(row[0], row[1], "mil")

Germany 36056.0 mil
United States 25764.0 mil
Japan 22952.0 mil
Italy 22306.0 mil
China 15189.0 mil
Spain 13673.0 mil
France 6392.0 mil
India 5020.0 mil
United Kingdom 4054.0 mil
Australia 4010.0 mil
Greece 3792.0 mil
Belgium 2886.0 mil
Korea, Republic of 2557.0 mil
Czechia 2123.0 mil
Canada 2120.0 mil


## Question 2: aggregate query

Using avg(), get the average solar energy usage for each country, and display only the countries with an average that is above 500 (million). (Remember for aggregates you need to use HAVING to test values, instead of WHERE). Make sure it is properly ordered like the results below, and wrap your avg column in round() to get round numbers:

```
69405.0 China
15402.0 India
11208.0 United States
10918.0 Germany
9098.0 Japan
5818.0 Italy
4122.0 Spain
2503.0 United Kingdom
2389.0 Turkey
2234.0 France
1705.0 Greece
1690.0 Chile
1607.0 Australia
1588.0 Brazil
1532.0 Belgium
1494.0 Thailand
1397.0 Czechia
1348.0 South Africa
1150.0 Korea, Republic of
906.0 Bulgaria
804.0 Honduras
789.0 Romania
773.0 Israel
740.0 Canada
652.0 Jordan
573.0 Pakistan
509.0 Other Asia
```

In [3]:
cursor = conn.cursor()
myquery = '''
SELECT ROUND(AVG(usage)), country FROM solar 
GROUP BY country
HAVING ROUND(AVG(usage)) > 500 
ORDER BY ROUND(AVG(usage)) DESC;
'''
cursor.execute(myquery)
for row in cursor:
    print(row[0],row[1])

69405.0 China
15402.0 India
11208.0 United States
10918.0 Germany
9098.0 Japan
5818.0 Italy
4122.0 Spain
2503.0 United Kingdom
2389.0 Turkey
2234.0 France
1705.0 Greece
1690.0 Chile
1607.0 Australia
1588.0 Brazil
1532.0 Belgium
1494.0 Thailand
1397.0 Czechia
1348.0 South Africa
1150.0 Korea, Republic of
906.0 Bulgaria
804.0 Honduras
789.0 Romania
773.0 Israel
740.0 Canada
652.0 Jordan
573.0 Pakistan
509.0 Other Asia


## Question 3: aggregate query 2

Get a list of countries whose Solar Energy data first started being collected in 2012. This is tricky to figure out, but the query should be pretty simple. The main thing you need to figure out is which aggregate function will find the earliest date, and then you need to test if year is 2012. And make it alphabetical by country.


```
2012 Botswana
2012 Fiji
2012 Gabon
2012 Lao People's Dem. Rep.
2012 Lithuania
2012 Marshall Islands
2012 Mauritius
2012 Nigeria
2012 Poland
2012 South Sudan
2012 St. Helena and Depend.
2012 St. Kitts-Nevis
2012 Tuvalu
2012 United Arab Emirates
```


In [4]:
cursor = conn.cursor()
myquery = '''
SELECT MIN(year), country FROM solar 
GROUP BY country 
HAVING MIN(year) = 2012 
ORDER BY country ASC;
'''
cursor.execute(myquery)
for row in cursor:
    print(row[0], row[1])

2012 Botswana
2012 Fiji
2012 Gabon
2012 Lao People's Dem. Rep.
2012 Lithuania
2012 Marshall Islands
2012 Mauritius
2012 Nigeria
2012 Poland
2012 South Sudan
2012 St. Helena and Depend.
2012 St. Kitts-Nevis
2012 Tuvalu
2012 United Arab Emirates


## Question 4: JOIN

Get the usage of wind AND solar power for Germany for each year in the tables. There should be four columns total, and you will be joining on two criteria.

(Please note that in the results below I am using Python to add in the strings "solar:" and "wind:")



```
Germany 1991 	wind: 215.0 	solar: 1.0
Germany 1992 	wind: 291.0 	solar: 4.0
Germany 1993 	wind: 674.0 	solar: 3.0
Germany 1994 	wind: 1428.0 	solar: 7.0
Germany 1995 	wind: 1712.0 	solar: 7.0
Germany 1996 	wind: 2078.0 	solar: 12.0
Germany 1997 	wind: 3034.0 	solar: 18.0
Germany 1998 	wind: 4593.0 	solar: 35.0
Germany 1999 	wind: 5528.0 	solar: 30.0
Germany 2000 	wind: 9352.0 	solar: 60.0
Germany 2001 	wind: 10456.0 	solar: 116.0
Germany 2002 	wind: 15856.0 	solar: 188.0
Germany 2003 	wind: 18713.0 	solar: 313.0
Germany 2004 	wind: 25509.0 	solar: 557.0
Germany 2005 	wind: 27229.0 	solar: 1282.0
Germany 2006 	wind: 30710.0 	solar: 2220.0
Germany 2007 	wind: 39713.0 	solar: 3075.0
Germany 2008 	wind: 40574.0 	solar: 4420.0
Germany 2009 	wind: 38647.0 	solar: 6584.0
Germany 2010 	wind: 38547.0 	solar: 11729.0
Germany 2011 	wind: 49858.0 	solar: 19599.0
Germany 2012 	wind: 51680.0 	solar: 26380.0
Germany 2013 	wind: 52737.0 	solar: 31010.0
Germany 2014 	wind: 58497.0 	solar: 36056.0
Germany 2015 	wind: 80624.0 	solar: 38726.0
Germany 2016 	wind: 79924.0 	solar: 38098.0
Germany 2017 	wind: 105693.0 	solar: 39401.0
Germany 2018 	wind: 109951.0 	solar: 45784.0
```

Hint: after the join you need to test that the country is Germany, then order by what?

In [5]:
cursor = conn.cursor()
myquery = '''
SELECT solar.country, solar.year, wind.usage, solar.usage FROM solar
JOIN wind ON solar.country = wind.country AND solar.year = wind.year 
WHERE solar.country = 'Germany' 
ORDER BY solar.year ASC;
'''
cursor.execute(myquery)
for row in cursor:
    print(row[0],row[1],"\twind:",row[2],"\tsolar:",row[3])

Germany 1991 	wind: 215.0 	solar: 1.0
Germany 1992 	wind: 291.0 	solar: 4.0
Germany 1993 	wind: 674.0 	solar: 3.0
Germany 1994 	wind: 1428.0 	solar: 7.0
Germany 1995 	wind: 1712.0 	solar: 7.0
Germany 1996 	wind: 2078.0 	solar: 12.0
Germany 1997 	wind: 3034.0 	solar: 18.0
Germany 1998 	wind: 4593.0 	solar: 35.0
Germany 1999 	wind: 5528.0 	solar: 30.0
Germany 2000 	wind: 9352.0 	solar: 60.0
Germany 2001 	wind: 10456.0 	solar: 116.0
Germany 2002 	wind: 15856.0 	solar: 188.0
Germany 2003 	wind: 18713.0 	solar: 313.0
Germany 2004 	wind: 25509.0 	solar: 557.0
Germany 2005 	wind: 27229.0 	solar: 1282.0
Germany 2006 	wind: 30710.0 	solar: 2220.0
Germany 2007 	wind: 39713.0 	solar: 3075.0
Germany 2008 	wind: 40574.0 	solar: 4420.0
Germany 2009 	wind: 38647.0 	solar: 6584.0
Germany 2010 	wind: 38547.0 	solar: 11729.0
Germany 2011 	wind: 49858.0 	solar: 19599.0
Germany 2012 	wind: 51680.0 	solar: 26380.0
Germany 2013 	wind: 52737.0 	solar: 31010.0
Germany 2014 	wind: 58497.0 	solar: 36056.0
Germa

## Question 5: join with aggregates

Get the average wind usage and solar usage for each country. Order your results by average solar usage, and show only the top ten countries.

```
China wind: 162851.40 	solar: 69405.33
India wind: 13008.79 	solar: 15401.83
United States wind: 67514.24 	solar: 11208.00
Germany wind: 32279.39 	solar: 10918.39
Japan wind: 2266.03 	solar: 9098.38
Italy wind: 5464.52 	solar: 5818.48
Spain wind: 22075.00 	solar: 4122.48
United Kingdom wind: 11362.24 	solar: 2503.00
Turkey wind: 4655.62 	solar: 2388.60
France wind: 7280.31 	solar: 2234.04
```

In [5]:
cursor = conn.cursor()
myquery = '''
SELECT solar.country, AVG(wind.usage), AVG(solar.usage) FROM solar 
JOIN wind ON solar.country = wind.country 
GROUP BY solar.country 
ORDER BY AVG(solar.usage) DESC LIMIT 10;
'''
cursor.execute(myquery)
for row in cursor:
    print(row[0],"wind:","%.2f" % row[1],"\tsolar:","%.2f" % row[2])

China wind: 162851.40 	solar: 69405.33
India wind: 13008.79 	solar: 15401.83
United States wind: 67514.24 	solar: 11208.00
Germany wind: 32279.39 	solar: 10918.39
Japan wind: 2266.03 	solar: 9098.38
Italy wind: 5464.52 	solar: 5818.48
Spain wind: 22075.00 	solar: 4122.48
United Kingdom wind: 11362.24 	solar: 2503.00
Turkey wind: 4655.62 	solar: 2388.60
France wind: 7280.31 	solar: 2234.04


## Question 6: same thing but slightly more complicated

Get the average wind usage and solar usage for each country, FROM 2010 TO THE PRESENT. Order your results by average solar usage, and show only the top ten countries.

```
China wind: 177957.11 	solar: 69405.33
United States wind: 185230.56 	solar: 33785.67
Germany wind: 69723.44 	solar: 31864.78
Japan wind: 5517.00 	solar: 27680.67
Italy wind: 14495.00 	solar: 18615.22
India wind: 33272.00 	solar: 15401.83
Spain wind: 49174.78 	solar: 12218.33
France wind: 18550.56 	solar: 6170.56
United Kingdom wind: 32268.67 	solar: 5552.89
Australia wind: 9749.56 	solar: 4535.78
```

In [19]:
cursor = conn.cursor()
myquery = '''
WITH since2010 AS(
SELECT wind.country, wind.year, wind.usage AS wind, solar.usage AS solar FROM wind 
LEFT JOIN solar ON wind.year = solar.year AND wind.country = solar.country
WHERE wind.year >2009)
SELECT since2010.country, AVG(since2010.wind), AVG(since2010.solar) FROM since2010
GROUP BY since2010.country 
ORDER BY AVG(since2010.solar) DESC NULLS LAST LIMIT 10;
'''
cursor.execute(myquery)
for row in cursor:
    print(row[0],"wind:","%.2f" % row[1],"\tsolar:","%.2f" % row[2])

China wind: 177957.11 	solar: 69405.33
United States wind: 185230.56 	solar: 33785.67
Germany wind: 69723.44 	solar: 31864.78
Japan wind: 5517.00 	solar: 27680.67
Italy wind: 14495.00 	solar: 18615.22
India wind: 33272.00 	solar: 15401.83
Spain wind: 49174.78 	solar: 12218.33
France wind: 18550.56 	solar: 6170.56
United Kingdom wind: 32268.67 	solar: 5552.89
Australia wind: 9749.56 	solar: 4535.78


## Question 7: Building a basic timeseries

Step one: get a list of the first year of data and the most recent year of data for each country from the solar table. Limit the results to the first 15 with the oldest year and then in alphabetical order.

```
Italy 1990 -- 2018
Japan 1990 -- 2018
Korea, Republic of 1990 -- 2018
Mexico 1990 -- 2018
Portugal 1990 -- 2018
Spain 1990 -- 2018
Switzerland 1990 -- 2018
United States 1990 -- 2018
Finland 1991 -- 2018
Germany 1991 -- 2018
Netherlands 1991 -- 2018
Canada 1992 -- 2018
Australia 1993 -- 2018
Austria 1993 -- 2018
Liechtenstein 1993 -- 2019
```

In [25]:
cursor = conn.cursor()
myquery = '''
SELECT country, MIN(year), MAX(year) FROM solar 
GROUP BY country
ORDER BY MIN(year), country ASC LIMIT 15;
'''
cursor.execute(myquery)
for row in cursor:
    print(row[0],row[1],"--",row[2])

Italy 1990 -- 2018
Japan 1990 -- 2018
Korea, Republic of 1990 -- 2018
Mexico 1990 -- 2018
Portugal 1990 -- 2018
Spain 1990 -- 2018
Switzerland 1990 -- 2018
United States 1990 -- 2018
Finland 1991 -- 2018
Germany 1991 -- 2018
Netherlands 1991 -- 2018
Canada 1992 -- 2018
Australia 1993 -- 2018
Austria 1993 -- 2018
Liechtenstein 1993 -- 2019


## Question 8: Building a basic timeseries

Step two: Take that previous query, (order, limit and everything) and turn that into a subquery, and join the solar table on it twice, first to get the usage from the earliest date, next get the usage from the most recent date. Order it by the most usage from the most recent year.

Note: to join at the same table twice, you need to alias the table (solar as sl1, solar as sl2). This is getting very challenging, so get as far as you can.
```
United States 1990 -- 2018 usage change:  666.0 -- 85184.0
Japan 1990 -- 2018 usage change:  67.0 -- 62668.0
Germany 1991 -- 2018 usage change:  1.0 -- 45784.0
Italy 1990 -- 2018 usage change:  4.0 -- 22654.0
Spain 1990 -- 2018 usage change:  9.0 -- 12744.0
Australia 1993 -- 2018 usage change:  11.0 -- 9929.0
Korea, Republic of 1990 -- 2018 usage change:  1.0 -- 9208.0
Canada 1992 -- 2018 usage change:  2.0 -- 3796.0
Netherlands 1991 -- 2018 usage change:  1.0 -- 3693.0
Switzerland 1990 -- 2018 usage change:  1.0 -- 1944.0
Mexico 1990 -- 2018 usage change:  1.0 -- 1856.0
Austria 1993 -- 2018 usage change:  1.0 -- 1438.0
Portugal 1990 -- 2018 usage change:  1.0 -- 1006.0
Finland 1991 -- 2018 usage change:  1.0 -- 90.0
Liechtenstein 1993 -- 2019 usage change:  0.008 -- 26.0
```

In [18]:
cursor = conn.cursor()
myquery = '''
WITH time AS(SELECT country, MIN(year), MAX(year) FROM solar 
GROUP BY country
ORDER BY MIN(year), country ASC LIMIT 15)
SELECT time.country, time.min, time.max, s1.usage, s2.usage FROM time 
JOIN solar AS s1 ON time.country = s1.country AND time.min = s1.year
JOIN solar AS s2 ON time.country = s2.country AND time.max = s2.year
ORDER BY s2.usage DESC;
'''  
cursor.execute(myquery)
for row in cursor:
    print(row[0],row[1],"--",row[2],"usage change: ", row[3],"--",row[4])

United States 1990 -- 2018 usage change:  666.0 -- 85184.0
Japan 1990 -- 2018 usage change:  67.0 -- 62668.0
Germany 1991 -- 2018 usage change:  1.0 -- 45784.0
Italy 1990 -- 2018 usage change:  4.0 -- 22654.0
Spain 1990 -- 2018 usage change:  9.0 -- 12744.0
Australia 1993 -- 2018 usage change:  11.0 -- 9929.0
Korea, Republic of 1990 -- 2018 usage change:  1.0 -- 9208.0
Canada 1992 -- 2018 usage change:  2.0 -- 3796.0
Netherlands 1991 -- 2018 usage change:  1.0 -- 3693.0
Switzerland 1990 -- 2018 usage change:  1.0 -- 1944.0
Mexico 1990 -- 2018 usage change:  1.0 -- 1856.0
Austria 1993 -- 2018 usage change:  1.0 -- 1438.0
Portugal 1990 -- 2018 usage change:  1.0 -- 1006.0
Finland 1991 -- 2018 usage change:  1.0 -- 90.0
Liechtenstein 1993 -- 2019 usage change:  0.008 -- 26.0


## Question 9: join with aggregates, new data

Your final task: go back to the [UN data site](http://data.un.org/Explorer.aspx) and download another table of your choice from the energy statistics. [Full instructions here.](http://floatingmedia.com/columbia/csv_to_sql.html) Choose a table that has similar columns to the wind and energy tables, and input it into your database.  Make sure you test that everything is working in the shell before going back to Python.

Now, write a query that joins the third table to the query you made in Question 5. Get the average use for each Energy type: Solar, Wind, and the table you chose. Order it by the energy type you chose (that column should come first after the country). And display the top 10 countries by the average of the energy type that you added, along with the averages for solar and wind.


In [26]:
cursor = conn.cursor()
myquery = '''
WITH solarwater AS (
SELECT fallingwater.country, AVG(fallingwater.usage) AS wateru, AVG(solar.usage) AS solaru FROM fallingwater 
FULL JOIN solar ON fallingwater.country = solar.country
GROUP BY fallingwater.country),
winda AS (SELECT country, AVG(usage) AS windu FROM wind
GROUP BY country)
SELECT solarwater.country, solarwater.wateru, solarwater.solaru, winda.windu FROM solarwater
FULL JOIN winda ON solarwater.country = winda.country
WHERE solarwater.wateru IS NOT NULL AND solarwater.solaru IS NOT NULL
ORDER BY winda.windu DESC NULLS LAST LIMIT 10;
'''
cursor.execute(myquery)
for row in cursor:
    print(row[0],"%.2f" % row[1],"%.2f" % row[2],"%.2f" % row[3])

China 5975756.67 69405.33 162851.40
United States 1075300.00 11208.00 67514.24
Germany 120303.00 10918.39 32279.39
Spain 145200.00 4122.48 22075.00
Brazil 3026866.67 1587.86 17953.08
India 2637900.00 15401.83 13008.79
United Kingdom 17583.33 2503.00 11362.24
Canada 1559662.00 739.63 7936.67
France 256333.33 2234.04 7280.31
Denmark 120.00 215.60 6254.55


Great work! You're done.